# Compare Two Different Datasets using Wilcoxon Rank Sum Test

### INFO

__author__    = 'Lindsay Thomas, Scott Kleinman'  
__copyright__ = 'copyright 2020, The WE1S Project'  
__license__   = 'MIT'  
__version__   = '2.0'  
__email__     = 'lindsaythomas@miami.edu'

This noteboook allows you to compare two sets of textual data to one another to discover how, and how much, they differ based on word frequency. The notebook performs this comparison using the Wilcoxon rank sum test, a statistical test that determines if two samples (i.e., the relative frequencies of a specific word in two different datasets) are taken from populations that are significantly different from one another (meaning, if they have different distributions). When used to compare word frequency data from two different datasets, it helps you to determine what the most "significant" words in each dataset are.

For details on the Wilcoxon rank sum test, see <a href="https://data.library.virginia.edu/the-wilcoxon-rank-sum-test/" target="_blank">this description</a> by the University of Virgina Library. For implementations of the Wilcoxon rank sum test in literary studies, see Andrew Piper and Eva Portelace's article <a href="http://post45.org/2016/05/how-cultural-capital-works-prizewinning-novels-bestsellers-and-the-time-of-reading/" target="_blank">How Cultural Capital Works: Prizewinning Novels, Bestsellers, and the Time of Reading</a> (<em>Post45</em>, 5.10.16)</A>, and chapter 4 "Fictionality" from Andrew Piper's book <em>Enumerations</em>.

## Settings

This code block imports required Python modules and the `compare_word_frequencies.py` script. It also sets some required filepaths needed for analysis.

In [ ]:
# Python imports
import os
import csv
from pathlib import Path
from IPython.display import display, HTML
import pandas as pd
import qgrid

# Import scripts
%run scripts/compare_word_frequencies.py

# Define paths
current_dir      = %pwd
current_pathobj  = Path(current_dir)
project_dir      = str(current_pathobj.parent.parent)
compare_results  = current_dir + '/results'

display(HTML('<p style="color: green;"><strong>Setup complete.</strong></p>'))

## Workflow

**Prior to beginning the workflow**, you already must have two `doc-terms` files, one for each dataset you wish to compare. A `doc-terms` file is a text file representing the vocabulary in each document in the dataset. `doc-terms` files must contain rows of space-delimited columns beginning with the filename and index number (starting with 0), and then a list of every token in the document (with each instance of a token in its own column). Each row in each file should represent one document. A small example is given below:

```csv
sample_document1.json 0 21st a a a an an absolutely abyss academic addition advance afghanistan...
sample_document2.json 1 a a a an an an artificial artificial food food ingredients...
```

As you can see in the example above, the data in these files should already be tokenized and (ideally) stripped of stop words. If you have already prepared your data for topic modeling, there will be a `doc_terms.txt` file in your project's `project_data/models`, which you can use as an example of the format. This notebook does not include any processes for tokenizing or stripping stop words from document data.

**Important: The two `doc-terms` files you are comparing must not have overlapping filenames.**

The cells below are divided into three sections.

#### Section 1

In **Section 1A**, you will configure the names of the two `doc-terms` files to be compared. Once you have performed this step, you can proceed to **Section 2**, or you can go directly to **Section 2** and configure your `doc-terms` files there. Before doing so, you can use **Section 1B** and/or **Section 1C** to generate new `doc_terms` files containing subsets of the data in your original `doc-terms` file based on a list of filenames (**Section 1B**) or a random sample (**Section 1C**).

#### Section 2

In **Section 2**, you will obtain word counts and relative frequency data from your two data sets. At the beginning of the section you can check your current dataset files, overriding the settings in **Section 1A** if you have run that cell. You have the option to filter low-frequency words and to preview your data before generating a vocabulary list to use in the Wilcoxon rank sum test. The cells in all subsections must be run, except subsections marked optional.

#### Section 3

**You must have run Section 2 in order to proceed to Section 3.** Here you will configure the files generated in **Section 2** be used in performing the Wilcoxon rank sum test, and then you will run the test. The last cell provides you the option to explore the results in an interactive table.

<hr style="height:3px;">

## 1. Select Data

Select the datasets you wish to compare. If you wish to use the full versions of your `doc-terms` files, you may run **Section 1A** only and then skip to **Section 2**. Alternatively, you can go directly to **Section 2** and configure the datasets there.

If you wish to use portions of your datasets, run **Section 1A** and then **Section 1B** and/or **Section 1C**.

#### Notes

1. Your two datasets should be roughly equally sized. If they are not, consider using **Section 1B** or **Section 1C** to reduce the size of one of the datasets.
2. The code in this notebook is not optimized for large collections of data. If the datasets you want to compare contain over about 4000 documents each, we recommend that you use **Section 1C** to create randomly selected smaller samples of your data.

### A) Use Already Existing Files for Comparison

Configure the filepaths to the two `doc-terms` files you wish to compare.

In [ ]:
# Filepaths to the two doc-terms files you wish to compare
docterms_c1 = '' # E.g. 'data/hum-docterms-500.txt'
docterms_c2 = '' # E.g. 'data/sci-docterms-500.txt'

display(HTML('<p style="color: green;">Filepaths configured. Sections B and C below are optional.</p>'))

### B) Select only Specific Documents from an Existing `doc-terms` File (Optional)

If you wish to use only a selection of documents from the full `doc-terms` file, create a new .txt file with the filenames of each document on a separate line. It should look like this:

```
filename1.json
filename2.json
etc.
```

Once you have created this file, proceed to the other cells in this section. Remember that you may wish to do this for both files you are comparing.

#### Configure the Filenames

Set the path to the list of filenames, the path to the original `doc-terms` file, and the path to the output `doc-terms` file containing only the documents selected.

In [ ]:
# Filepath to the list of filenames
filenames = '' # E.g. 'data/hum-filenames-400.txt'

# Filepath to the original doc-terms file containing the files in the list
docterms_original = '' # E.g. 'data/hum-docterms-500.txt'

# Filepath to the doc-terms file the below code will create including only the selected files
docterms_selected = '' # E.g. 'hum-docterms-400.txt'

display(HTML('<p style="color: green;">Filenames configured.</p>'))

#### Create the `doc-terms` File

Run the cell below to create your new `doc-terms` file containing only the selected documents.

In [ ]:
get_bags(filenames, docterms_original, docterms_selected)
display(HTML('<p>Section C below is optional.</p>'))

### C) Randomly Select Sample Data from an Existing `doc-terms.txt` File (Optional)

If you wish to use a random selection of documents from the full `doc-terms` file, run the cells below.

#### Configure Selection and Filenames

First, set the `selection` variable to the number of documents you want to select. Please follow this format exactly: `selection = 1500`.

Next configure the path to the original `doc-terms` file and the path to the output `doc-terms` file containing only the documents in the random sample. You may change the values of the variables later in the notebook to reflect your filenames if needed.

In [ ]:
# Select the number of documents in the sample.
selection = 200

# Filepath to the original `doc-terms` file
docterms_original = '' # E.g. 'data/sci-docterms-500.txt'

# Filepath to the `doc-terms` file you will create including only the randomly sampled documents
docterms_selected = '' # E.g. 'data/sci-docterms-200.txt'

display(HTML('<p style="color: green;">Selection and filenames configured.</p>'))

#### Generate a New `doc-terms` File with the Random Sample

In [ ]:
get_random_sample(selection, docterms_original, docterms_selected)

<hr style="height:3px;">

## 2. Prepare Data for Test

In this section, you will find word frequencies for the datasets in your `doc-terms` files. You will have the opportunity below to check that the paths to your `doc-terms` files are configured correctly.

### Check Paths to `doc-terms` Files

Make sure to set the `docterms_c1` and `docterms_c2` variables to your desired files by running the cell below. If the values that print out below are missing or incorrect, uncomment the lines above to set the correct filenames.

In [ ]:
# Uncomment if you need to reset filenames
# docterms_c1 = '' # E.g. 'data/hum-docterms-200.txt'
# docterms_c2 = '' # E.g. 'data/hum-docterms-200.txt'

for file in [('First', docterms_c1, 'docterms_c1'), ('Second', docterms_c2, 'docterms_c2')]:
    try:
        display(HTML('<p>' + file[0] + ' doc-terms file: <code>' + file[1] + '</code></p>'))
    except NameError:
        display(HTML('<p style="color:#FF0000";>The ' + file[2] + ' variable not configured.</p>'))

### Get Raw and Relative Frequencies

The cell below finds the raw and relative frequencies for each word in your two datasets.

In [ ]:
df1_r, df1_f = findFreq(docterms_c1)
display(HTML('<p style="color: green;">Completed first dataset.</p>'))

df2_r, df2_f = findFreq(docterms_c2)
display(HTML('<p style="color: green;">Completed second dataset.</p>'))

#### Calculate Totals and Sort Dataframes

This cell adds a `total_count` column to the raw frequency dataframes listing the total number of times each word appears in each dataframe. It also sorts the raw frequency dataframes by highest to lowest total count values to find the top words in each dataset.

In [ ]:
df1_relative, df1_freqs, df2_relative, df2_freqs = edit_freq_dataframes(df1_r, df1_f, df2_r, df2_f)

#### Remove Infrequent Words

Some words may seem "distinctive" to each dataset only because they occur infrequently. This cell allows you to set a threshold for eliminating low-frequency words.

Based on what your average total word counts are as reported in the cell above, set your threshold for inclusion. The `threshold` variable represents the number of total times any word must appear in either dataset in order to be included. For instance, if you set `threshold=5` (the default), this means that any words that do not occur at least 5 times in either dataset will not be included in your analysis, while any words that do occur at least 5 times (or more) will be included in your analysis. 

If you wish to change the threshold number, configure it in the cell below. If you wish to **not** set a threshold for inclusion, set `threshold=False` before running the cell.

In [ ]:
threshold = 5

display(HTML('<p style="color: green;">Threshold is configured to <code>' + str(threshold) + '</code>.</p>'))

Now use this threshold value to create new dataframes including only those words that occur at least x number of times (where x = threshold). The output of this cell will tell you how many words are now in each of your datasets (i.e., how many words meet the threshold you set). If you didn't set a threshold value above you still need to configure and run the below cell to get variable names set right and produce the csv files you need to continue.

The code below will save your newly created dataframes to disk so that you can reload them if you want to run this test again. It will produce 2 csv files for each dataset: a csv of relative frequency counts for each word that meets the threshold for inclusion, and a csv of raw frequency counts for each word that meets the threshold for inclusion.

Change the variables below to reflect your desired filenames. Please follow this format exactly for each variable:

`c1_relative_csv = 'filepath-here.csv'`

In [ ]:
c1_relative_csv = '' # E.g. 'c1-relative.csv'
c2_relative_csv = '' # E.g. 'c2-relative.csv'
c1_raw_csv = '' # E.g. 'c1-raw.csv'
c2_raw_csv = '' # E.g. 'c2-raw.csv'

df1, df2, words_c1, words_c2 = match_dataframes_and_save(threshold, df1_freqs, df1_relative, df2_freqs, df2_relative,
                                                         c1_relative_csv, c2_relative_csv, c1_raw_csv, c2_raw_csv)

#### Preview Dataframes (Optional)

Use the cells below to preview the dataframes you just created. First, set `preview_csv` to the csv variable, configured above, that you want to preview. For example, if you want to preview the category 1 relative frequency csv, you would set it with  `preview_csv = c1_relative_csv`.

If you wanted to preview the category 2 raw frequency csv, you would set it with `preview_csv = c2_raw_csv`.

In [ ]:
preview_csv = c1_raw_csv

display(HTML('<p style="color: green;">Preview is configured to <code>' + preview_csv + '</code>.</p>'))

Next, run the below cell to preview the first 5 rows of this csv. What you will see is a dataframe where each row is a unique word in that category's dataset, and each column is a document in that category's dataset. The values in each cell tell you either the relative or raw frequency (depending on the csv you are previewing) for that word in that document. Keep in mind that these dataframes are sparse, meaning that the majority of the cell values are 0. The raw frequency dataframes, however, are sorted by the `total_count` column so that the most frequent words in terms of raw counts appear at the top.

The code below will preview only the first 5 rows of any dataframe (and only a handful of the columns). You can change the number of rows by entering a number like `df.head(20)` or use `df[10:20]` to view a range of rows.

It is not possible to see every row and column in the notebook environment. If you would like to examine the dataframes you have created in more detail, you should open them in Excel or another csv reader.

In [ ]:
df = pd.read_csv(preview_csv, index_col=0) 

df.head()

### Create the Vocabulary List for your Datasets

Finally, run the code below to create the vocabulary list for both of your datasets. This is a list of all of the words that appear at least x times in either of your datasets (where x = threshold). The function below will produce a plain text file listing each word that meets the threshold for inclusion in either of your datasets, with 1 word per line. Please follow the below format exactly to change the filename:

```
vocablist = 'vocablist-filename.txt'
```

If you have already produced a `vocablist` file with the same filename, the code below will delete that existing file and create a new one.

In [ ]:
vocablist = '' # E.g. 'vocablist.txt'

get_vocablist(df1, df2, words_c1, words_c2, vocablist)

<hr style="height:3px;">

## 3. Run Wilcoxon Rank Sum Test

You are now ready to perform a Wilcoxon rank sum test on your 2 datasets.

### Configuration

First, decide what you want to call the results file, which will be saved to a csv file in the `results` folder in this module. Please use this format to change the filename: `comparison_csv = compare_results + '/results-filename.csv'`.

In [ ]:
results_csv = compare_results + '/wilcoxon-results.csv'

display(HTML('<p style="color: green;">Test output file is configured to <code>' + results_csv + '</code>.</p>'))

### Check Variables

Check the variables you need to perform the test. If any of the variables refer to the wrong file, or haven't been set, you can redefine them in the cell below by uncommenting the lines at the top.

In [ ]:
# Uncomment to redefine variables
# csv_relative_c1 =
# csv_relative_c2 = 
# csv_raw_c1 = 
# csv_raw_c2 = 
# vocablist = 

# Check to see if variables are defined
var_names =  ['c1_relative_csv', 'c2_relative_csv', 'c1_raw_csv', 'c2_raw_csv', 'vocablist']
for i, variable in enumerate([c1_relative_csv, c2_relative_csv, c1_raw_csv, c2_raw_csv, vocablist]):
    try:
        display(HTML('<p><strong>' + var_names[i] +':</strong> <code>' + variable + '</code></p>'))
    except NameError:
        display(HTML('<p style="color:#FF0000";>The ' + variable + ' variable as not been set.</p>'))

### Run the Test

The code below will perform the test. The results will be saved in a csv file in the `results` folder in this module (which you named above).

In [ ]:
# Run Wilcoxon rank sum test
wrs_test(c1_relative_csv, c1_raw_csv, c2_relative_csv, c2_raw_csv, vocablist, results_csv)

### Explore your results (Optional)

You can use the cell below to explore your results csv. The results csv lists the raw count of each word in each dataset, the difference between those counts, the percentage change in counts from dataset 1 to dataset 2, the Wilcoxon statistic, and the p-value for each comparison. Sorting the dataframe by the 'wilcoxon statistic' column from highest to lowest will bring those words most distinctive to cateogry 1 to the top, while sorting the 'wilcoxon statistic' column from lowest to highest will bring those words most distinctive to category 2 to the top. Click the column titles to sort. The p-value gives you a sense of how confident you can be in the results of each comparison. A p-value under 0.05 is generally considered significant.

In [ ]:
df = pd.read_csv(results_csv, index_col=0) 

qgrid_widget = qgrid.show_grid(df, show_toolbar=False)
qgrid_widget